In [ ]:
import re
from collections import Counter
import random

import pandas as pd
import spacy
import torch
from radgraph import RadGraph
from rich import print
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Spacy's English tokenizer
nlp = spacy.load("en_core_web_sm")

# Initialize RadGraph
radgraph = RadGraph()

DATA_DIR = "C:/Users/DryLab/Desktop/villa/data/open-i"

c:\Users\DryLab\anaconda3\envs\ViLLA\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model_type not provided, defaulting to radgraph-xl


c:\Users\DryLab\anaconda3\envs\ViLLA\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
df = pd.read_feather(
    "C:/Users/DryLab/Desktop/ViLLA/data/open-i/annotations_region_final.feather"
)

In [3]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 19.0, 119.0, 145.0], [136.0, 19.0, 223...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 16.0, 99.0, 161.0], [117.0, 17.0, 219.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[45.0, 43.0, 109.0, 175.0], [122.0, 42.0, 200...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 13.0, 109.0, 192.0], [115.0, 12.0, 194...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[5.0, 34.0, 96.0, 177.0], [108.0, 32.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
...,...,...,...,...,...,...,...,...
3789,3789,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 11.0, 121.0, 174.0], [128.0, 14.0, 204...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
3790,3790,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 8.0, 111.0, 210.0], [119.0, 7.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
3791,3791,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 0.0, 107.0, 156.0], [116.0, 0.0, 196.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[37.0, 14.0, 116.0, 160.0], [122.0, 15.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...


In [4]:
text_path_list = df["text_filepath"].tolist()

In [5]:
def filter_sentences_by_absent_tokens(radgraph_output):
    remaining_sentences = []

    radgraph_output = radgraph(radgraph_output)

    # Iterate over RadGraph output
    for _, description in radgraph_output.items():
        text = description.get("text", " ")
        entities = description.get("entities", {})

        # Step 1: Split the text into sentences
        doc = nlp(text)
        sentences = [sent.text.lower() for sent in doc.sents]

        # Step 2: Create a mapping of word indices to sentence indices
        word_to_sentence = {}
        current_word_idx = 0

        for i, sentence in enumerate(sentences):
            sentence_words = sentence.split()
            for _ in sentence_words:
                word_to_sentence[current_word_idx] = i
                current_word_idx += 1

        # Step 3: Create a flag to keep or discard each sentence
        sentence_flags = [True] * len(sentences)

        # Step 4: Iterate over entities and check for tokens labeled "Observation::definitely absent"
        for _, entity_info in entities.items():
            label = entity_info.get("label")
            start_ix = entity_info.get("start_ix")

            # Check if the entity is labeled as "Observation::definitely absent"
            if label == "Observation::definitely absent":
                # Step 5: Find which sentence contains the token based on start and end indices
                sentence_idx = word_to_sentence.get(start_ix, None)
                if sentence_idx is not None:
                    sentence_flags[sentence_idx] = False

        # Step 6: Collect the remaining sentences
        for i, sentence in enumerate(sentences):
            if sentence_flags[i]:
                if not re.match(r"^\d\s*\.$", sentence.strip()):
                    remaining_sentences.append(sentence)

        # Revert back to the original text
        remaining_sentences = " ".join(remaining_sentences)

    return remaining_sentences

In [6]:
df["text"] = df["text_filepath"].apply(lambda x: open(x, "r").read())

In [7]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 19.0, 119.0, 145.0], [136.0, 19.0, 223...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,There is XXXX increased opacity within the rig...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 16.0, 99.0, 161.0], [117.0, 17.0, 219.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,There is XXXX increased opacity within the rig...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[45.0, 43.0, 109.0, 175.0], [122.0, 42.0, 200...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Interstitial markings are diffusely prominent ...
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 13.0, 109.0, 192.0], [115.0, 12.0, 194...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Status post left mastectomy. Heart size normal...
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[5.0, 34.0, 96.0, 177.0], [108.0, 32.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Heart size and pulmonary vascularity appear wi...
...,...,...,...,...,...,...,...,...,...
3789,3789,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 11.0, 121.0, 174.0], [128.0, 14.0, 204...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Cardiomediastinal silhouette demonstrates norm...
3790,3790,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 8.0, 111.0, 210.0], [119.0, 7.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Lungs are clear bilaterally. There is no focal...
3791,3791,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 0.0, 107.0, 156.0], [116.0, 0.0, 196.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Lungs are clear bilaterally. There is no focal...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[37.0, 14.0, 116.0, 160.0], [122.0, 15.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,Normal heart and mediastinum. Clear lungs. Tra...


In [8]:
df.text_filepath.iloc[2]

'C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i/data/ecgen-radiology-txt\\CXR1001_IM-0004-1001.txt'

In [ ]:
index = random.randint(0, len(df) - 1)

findings = df["text"].iloc[index]
# findings = "'the heart and lungs have [MASK] [MASK] in the interval . both lungs are clear and expanded . heart and mediastinum normal . no change right anterior soft tissue surgical clips . configuration of breast shadows on the pa view suggests prior right lumpectomy .'"
annotations = radgraph(findings)
filtered_text = filter_sentences_by_absent_tokens(findings)

print(f"Original Text: {findings}")
print(f"Preprocessed Text: {filtered_text}")

Original Text: The heart size is within normal limits. Cardiomediastinal contour is normal. There is a right upper 
lobe nodule measuring 8 mm in diameter. Trachea is midline. The lungs otherwise clear. XXXX and soft tissues are 
unremarkable. 1. Right upper lobe pulmonary nodule, XXXX granuloma.

Preprocessed Text: the heart size is within normal limits . cardiomediastinal contour is normal . there is a right 
upper lobe nodule measuring 8 mm in diameter . trachea is midline . the lungs otherwise clear . xxxx and soft 
tissues are unremarkable . right upper lobe pulmonary nodule , xxxx granuloma .

In [10]:
# Break down the text into sentences and filter out the sentences that contain tokens labeled as "Observation::definitely absent"
df["text"] = df["text"].apply(filter_sentences_by_absent_tokens)

In [11]:
index = random.randint(0, len(df) - 1)

print(f"Index {index}: {df.text.iloc[index]}")

Index 3640: the lungs appear clear . the heart and pulmonary xxxx appear normal . the pleural spaces are clear . 
these xxxx ' t contours appear normal . there is a xxxx fracture of the midthoracic vertebral body . the age of 
this fracture is unknown . there are healed fractures of several left anterior ribs . there is a healed left 
clavicle fracture . a single nonsclerotic mid thoracic vertebral xxxx fracture is present . without a comparison 
study , the age of this fracture is unknown . metastatic disease is possible , however given the nonsclerotic 
appearance , is not xxxx . healed left clavicle and left anterior rib fractures .

In [12]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 19.0, 119.0, 145.0], [136.0, 19.0, 223...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 16.0, 99.0, 161.0], [117.0, 17.0, 219.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is xxxx increased opacity within the rig...
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[45.0, 43.0, 109.0, 175.0], [122.0, 42.0, 200...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 13.0, 109.0, 192.0], [115.0, 12.0, 194...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[5.0, 34.0, 96.0, 177.0], [108.0, 32.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...
...,...,...,...,...,...,...,...,...,...
3789,3789,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 11.0, 121.0, 174.0], [128.0, 14.0, 204...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...
3790,3790,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 8.0, 111.0, 210.0], [119.0, 7.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...
3791,3791,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 0.0, 107.0, 156.0], [116.0, 0.0, 196.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[37.0, 14.0, 116.0, 160.0], [122.0, 15.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...


In [13]:
def clean_text(text):
    # Replace multiple consecutive periods ("..") with a single period (".")
    text = re.sub(r'\.\.', '.', text)
    # Replace standalone "X" or "x" with "[MASK]"
    text = re.sub(r"\b[xX]+\b", "[MASK]", text)
    return text

In [14]:
df["text"] = df["text"].apply(clean_text)

In [17]:
index = random.randint(0, len(df) - 1)
real_text = df.iloc[index]["text"]
findings = clean_text(df.iloc[index]["text"])
print(f"Original: {real_text}")
print(f"Masked: {findings}")

Original: normal heart size , mediastinal and aortic contours . normal pulmonary vascularity . elevated left 
hemidiaphragm with scarring at the left costophrenic [MASK] . there is a bullet fragment overlying the left t7 
vertebra . retained [MASK] bullet fragments noted within the left upper and lower lobes . mild degenerative changes
of the thoracic spine . posttraumatic changes compatible with prior gunshot wound .

Masked: normal heart size , mediastinal and aortic contours . normal pulmonary vascularity . elevated left 
hemidiaphragm with scarring at the left costophrenic [MASK] . there is a bullet fragment overlying the left t7 
vertebra . retained [MASK] bullet fragments noted within the left upper and lower lobes . mild degenerative changes
of the thoracic spine . posttraumatic changes compatible with prior gunshot wound .

In [22]:
def extract_entities(textual_descriptions):
    extracted_entities = []

    # RadGraph output processing
    textual_descriptions = radgraph(textual_descriptions) 
    
    for _, description in textual_descriptions.items():
        entities = description.get("entities", {})

        # Step 1: Identify all real entities (Observation::definitely present with no direct modify relations)
        all_entities = {
            entity_id: entity_data
            for entity_id, entity_data in entities.items()
            if entity_data.get("label") == "Observation::definitely present" or entity_data.get("label") == "Anatomy::definitely absent"
        }
        
        real_entities = {
            entity_id: entity_data
            for entity_id, entity_data in entities.items()
            if entity_data.get("label") == "Observation::definitely present" and not any("modify" in relation for relation in entity_data.get("relations", []))
        }

        # Step 2: Create a map of modifiers to the entities they modify
        modifier_map = {}
        for entity_id, entity_data in entities.items():
            relations = entity_data.get("relations", [])
            for relation in relations:
                if relation[0] == "modify" and relation[1] in entities:
                    modifier_map.setdefault(relation[1], []).append(entity_id)

        # Step 3: Recursive function to gather all modifiers for each real entity
        def get_all_modifiers(entity_id):
            visited = set()  # To track visited entities
            stack = [entity_id]  # Initialize stack with the starting entity
            modifiers = []  # List to store unique modifiers
            seen = set()  # To track unique (token, index) pairs

            while stack:
                current_entity_id = stack.pop()

                if current_entity_id in visited:
                    continue  # Skip already visited entities
                
                visited.add(current_entity_id)  # Mark current entity as visited

                # Add current entity's modifiers to the list
                direct_modifiers = modifier_map.get(current_entity_id, [])
                for modifier_id in direct_modifiers:
                    modifier_data = entities[modifier_id]
                    token_index_pair = (modifier_data.get("tokens"), modifier_data.get("start_ix"))

                    # Ensure unique modifiers
                    if token_index_pair not in seen:
                        modifiers.append(token_index_pair)
                        seen.add(token_index_pair)  # Mark as seen

                    stack.append(modifier_id)  # Add the modifier to the stack for further exploration

            return modifiers


        # Step 4: Combine real entities with all their modifiers
        for entity_id, entity_data in all_entities.items():
            observation_text = entity_data.get("tokens")
            # print(observation_text)
            observation_start_ix = entity_data.get("start_ix")
            # Gather all modifiers recursively
            modifiers_for_this_entity = get_all_modifiers(entity_id)
            # print(modifiers_for_this_entity)

            # Create a list of tokens, including both entity and modifier tokens
            all_tokens = [(observation_text, observation_start_ix)] + modifiers_for_this_entity

            # Sort all tokens by their start_ix to ensure correct order
            sorted_tokens = sorted(all_tokens, key=lambda x: x[1])

            # Extract the sorted tokens (ignoring the start_ix for final output)
            sorted_entity_text = " ".join([token[0] for token in sorted_tokens]).strip()
            # print(observation_text)

            # Ensure only real entities with their full modifier chains are added to the output
            if observation_text in real_entities.get(entity_id)['tokens'] if entity_id in real_entities else False:
                extracted_entities.append(sorted_entity_text)

    # Return unique extracted entities (no duplicates)
    return list(set(extracted_entities))

# index = random.randint(0, len(df) - 1)
findings = df.iloc[index]["text"]
# findings = "the lungs are clear bilaterally . large bilateral hilar calcified lymph [ MASK ] / granulomas . cardio mediastinal silhouette is unremarkable ."
# findings = "the heart size and pulmonary vascularity appear within normal limits . the left hemidiaphragm remains elevated . right base densities are again noted which appear improved . there continues to be some left base opacities which may represent atelectasis . surgical clips and suture lines are noted in the mediastinum . an air - fluid level is seen in the upper right abdomen immediately below the right hemidiaphragm . extensive pleural densities are present on the right which may represent localized fluid or pleural thickening . continued elevation of the left hemidiaphragm . extensive post - op changes . right pleural densities which may represent thickening or localized fluid . air - fluid level below the right hemidiaphragm medially . although this could represent air - fluid within a [ MASK ] of bowel , a subpulmonic process is a consideration . if clinically indicated , [ MASK ] scan would better define this"
# findings = "______________________________________________________________________________ final report examination : chest ( portable ap ) indication : ___ year old man s / p tracheostomy // eval for ptx and trach placement technique : chest ( portable ap ) comparison : ___ right central venous line tip is at the level of mid svc . right picc line is turning upwards at the junction of the right subclavian vein and right internal jugular vein and should be repositioned . right pigtail catheter is in place . bilateral pleural effusions are demonstrated ."
# findings = "the lungs are clear . the cardiomediastinal silhouette is within normal limits . normal chest film ."
textual_descriptions = findings  # Your RadGraph output dictionary
entities = extract_entities(textual_descriptions)
print(findings)
print(entities)

within the posterior lateral 8th rib there is a deformity along the cortex with associated oblique lucency . in 
addition within the posterior lateral 9th rib there appears to be a obliquely oriented lucency with cortical 
disruption . findings are concerning for possible left rib fractures . otherwise the cardiomediastinal silhouette 
is within normal limits . the lungs are clear bilaterally . multiple small punctate radiopaque foreign bodies are 
seen within the subcutaneous tissues and are present on previous ct scan from [MASK] . possible lower posterior 
lateral left rib fractures as described above . if further concern for rib fractures dedicated rib films would 
better evaluate .

[
    'obliquely',
    'findings',
    'oblique lucency',
    'normal',
    'multiple small punctate radiopaque foreign bodies',
    'deformity',
    'clear',
    'disruption',
    'oriented lucency'
]

In [43]:
def extract_entities(textual_descriptions):
    extracted_entities = []

    textual_descriptions = radgraph(textual_descriptions)

    for _, description in textual_descriptions.items():
        # Extract entities using RadGraph
        entities = description.get("entities", {})

        # Filter entities that are 'Definitely Present'
        real_entities = {
            entity_id: entity_data
            for entity_id, entity_data in entities.items()
            if entity_data.get("label") == "Observation::definitely present" and not any("modify" in relation for relation in entity_data.get("relations", []))
        }
        # print(real_entities)
        
        # Append real entities to the extracted entities list
        for entity_id, entity_data in real_entities.items():
            extracted_entities.append(entity_data.get("tokens"))
            # print(entity_data.get("tokens"))

    return extracted_entities


textual_descriptions = findings
entities = extract_entities(textual_descriptions)
print(entities)

['deformity', 'lucency', 'obliquely', 'lucency', 'disruption', 'findings', 'normal', 'clear', 'bodies']

In [186]:
# def filter_nouns(entities):
#     noun_entities = []

#     for entity in entities:
#         # Tokenize the entity using Spacy
#         doc = nlp(entity)

#         # Extract noun phrases rather than individual nouns
#         noun_phrases = [chunk.text for chunk in doc.noun_chunks]

#         # Append the noun phrases to the list
#         noun_entities.extend(noun_phrases)

#     return noun_entities


# noun_entities = filter_nouns(entities)
# print(noun_entities)

In [44]:
# Function to filter nouns
def filter_nouns(attribute_list):
    nouns_only = []
    for attribute in attribute_list:
        # Process the attribute with spaCy
        doc = nlp(attribute)
        # Check each token's part of speech
        if any(token.pos_ in {"NOUN", "PROPN"} for token in doc):  # NOUN for common nouns, PROPN for proper nouns
            nouns_only.append(attribute)
    return nouns_only

# Get only nouns
noun_attributes = filter_nouns(entities)

print(noun_attributes)


['deformity', 'lucency', 'lucency', 'disruption', 'findings', 'bodies']

In [45]:
df["attributes"] = df["text"].apply(extract_entities)

c:\Users\DryLab\anaconda3\envs\ViLLA\lib\site-packages\radgraph\dygie\data\dataset_readers\dygie.py:192: UserWarning: Document 0 has a sentence with a single token or no tokens. This may break the modeling code.
  warnings.warn(msg)


In [46]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 19.0, 119.0, 145.0], [136.0, 19.0, 223...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is [MASK] increased opacity within the r...,"[opacity, normal, opacity, opacity, atelectasi..."
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 16.0, 99.0, 161.0], [117.0, 17.0, 219.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is [MASK] increased opacity within the r...,"[opacity, normal, opacity, opacity, atelectasi..."
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[45.0, 43.0, 109.0, 175.0], [122.0, 42.0, 200...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,"[markings, normal, normal, fibrosis]"
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 13.0, 109.0, 192.0], [115.0, 12.0, 194...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...,"[mastectomy, normal, clear]"
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[5.0, 34.0, 96.0, 177.0], [108.0, 32.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[normal, density, this, hiatal hernia, calcifi..."
...,...,...,...,...,...,...,...,...,...,...
3789,3789,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 11.0, 121.0, 174.0], [128.0, 14.0, 204...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,"[normal, tortuosity, atherosclerosis, disease]"
3790,3790,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 8.0, 111.0, 210.0], [119.0, 7.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,"[clear, normal, unremarkable]"
3791,3791,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 0.0, 107.0, 156.0], [116.0, 0.0, 196.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,"[clear, normal, unremarkable]"
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[37.0, 14.0, 116.0, 160.0], [122.0, 15.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,"[normal, clear, midline, radiopaque, foreign b..."


In [47]:
df["attributes"] = df["attributes"].apply(filter_nouns)

In [48]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 19.0, 119.0, 145.0], [136.0, 19.0, 223...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is [MASK] increased opacity within the r...,"[opacity, opacity, opacity, atelectasis, opacity]"
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 16.0, 99.0, 161.0], [117.0, 17.0, 219.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is [MASK] increased opacity within the r...,"[opacity, opacity, opacity, atelectasis, opacity]"
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[45.0, 43.0, 109.0, 175.0], [122.0, 42.0, 200...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,"[markings, fibrosis]"
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 13.0, 109.0, 192.0], [115.0, 12.0, 194...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...,[mastectomy]
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[5.0, 34.0, 96.0, 177.0], [108.0, 32.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[density, hiatal hernia, calcification, granul..."
...,...,...,...,...,...,...,...,...,...,...
3789,3789,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 11.0, 121.0, 174.0], [128.0, 14.0, 204...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,"[tortuosity, atherosclerosis, disease]"
3790,3790,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 8.0, 111.0, 210.0], [119.0, 7.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[]
3791,3791,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 0.0, 107.0, 156.0], [116.0, 0.0, 196.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,lungs are clear bilaterally . cardiomediastina...,[]
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[37.0, 14.0, 116.0, 160.0], [122.0, 15.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,"[midline, radiopaque, foreign body]"


In [49]:
# # Remove row if no attributes are present
mask = df["attributes"].apply(lambda x: len(x) > 0)

df["attributes"] = df["attributes"].where(mask)

df = df.dropna(subset=["attributes"])

In [50]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[22.0, 19.0, 119.0, 145.0], [136.0, 19.0, 223...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is [MASK] increased opacity within the r...,"[opacity, opacity, opacity, atelectasis, opacity]"
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 16.0, 99.0, 161.0], [117.0, 17.0, 219.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,there is [MASK] increased opacity within the r...,"[opacity, opacity, opacity, atelectasis, opacity]"
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[45.0, 43.0, 109.0, 175.0], [122.0, 42.0, 200...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,interstitial markings are diffusely prominent ...,"[markings, fibrosis]"
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 13.0, 109.0, 192.0], [115.0, 12.0, 194...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,status post left mastectomy . heart size norma...,[mastectomy]
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[5.0, 34.0, 96.0, 177.0], [108.0, 32.0, 200.0...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size and pulmonary vascularity appear wi...,"[density, hiatal hernia, calcification, granul..."
...,...,...,...,...,...,...,...,...,...,...
3786,3786,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[12.0, 29.0, 117.0, 191.0], [130.0, 28.0, 216...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,sternotomy sutures and bypass grafts have been...,[bypass grafts]
3788,3788,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 21.0, 113.0, 167.0], [121.0, 22.0, 198...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,calcified mediastinal [MASK] . heart size with...,"[normal limits, degenerative changes]"
3789,3789,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 11.0, 121.0, 174.0], [128.0, 14.0, 204...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,cardiomediastinal silhouette demonstrates norm...,"[tortuosity, atherosclerosis, disease]"
3792,3792,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[37.0, 14.0, 116.0, 160.0], [122.0, 15.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,normal heart and mediastinum . clear lungs . t...,"[midline, radiopaque, foreign body]"


In [ ]:
# def update_attribute_list(attribute_list):
#     return ['within normal limits' if 'limits' in term.lower() else term for term in attribute_list]

# # Apply the function to the 'attributes' column
# df['attributes'] = df['attributes'].apply(update_attribute_list)

In [51]:
# Split the data into training and test sets (70% training and 30% test)
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)

# Split the remaining data into validation and test sets (10% validation and 20% test)
# val_df, test_df = train_test_split(
#     split_df, test_size=2 / 3, random_state=42, shuffle=True
# )

print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}")

Train size: 1596, Validation size: 684

In [52]:
train_df["split"] = "train"
val_df["split"] = "val"

In [53]:
attributes_token = [item for sublist in train_df["attributes"] for item in sublist]

In [54]:
# Check attributes named "normal limits"
df[df["attributes"].apply(lambda x: "within normal limits" in x)].iloc[0]['text']

'cardiac and mediastinal contours are within normal limits . prior granulomatous disease . the lungs are otherwise clear . thoracic spondylosis . bilateral breast prostheses with [MASK] calcification .'

In [55]:
count_attributes = Counter(attributes_token)

In [56]:
most_common_attributes = count_attributes.most_common(50)

In [57]:
most_common_attributes

[('cardiomegaly', 215),
 ('atelectasis', 205),
 ('opacities', 203),
 ('disease', 178),
 ('opacity', 167),
 ('limits', 137),
 ('normal limits', 136),
 ('granuloma', 106),
 ('degenerative changes', 97),
 ('effusions', 69),
 ('effusion', 67),
 ('emphysema', 65),
 ('changes', 60),
 ('markings', 58),
 ('granulomas', 58),
 ('calcifications', 54),
 ('midline', 50),
 ('density', 50),
 ('tortuosity', 48),
 ('sternotomy', 44),
 ('hiatal hernia', 43),
 ('lymph', 43),
 ('prominence', 43),
 ('deformity', 37),
 ('calcification', 37),
 ('osteophytes', 34),
 ('elevation', 34),
 ('edema', 33),
 ('surgical clips', 32),
 ('congestion', 31),
 ('nodules', 30),
 ('pneumonia', 28),
 ('enlargement', 26),
 ('clips', 23),
 ('hyperexpansion', 21),
 ('scoliosis', 21),
 ('copd', 21),
 ('dextroscoliosis', 20),
 ('calcified granuloma', 20),
 ('eventration', 19),
 ('pneumothorax', 18),
 ('tip', 18),
 ('consolidation', 17),
 ('aeration', 17),
 ('lucency', 17),
 ('ectasia', 16),
 ('deformities', 15),
 ('mild degenerati

In [58]:
df[df['attributes'].apply(lambda x: 'within normal limits' in x)].iloc[0]['attributes']

['within normal limits', 'disease', 'prostheses']

In [209]:
rad = radgraph(df[df['attributes'].apply(lambda x: 'within normal limits' in x)].iloc[0]['text'])
rad

{'0': {'text': 'there is [ MASK ] increased opacity within the right upper lobe with possible mass and associated area of atelectasis or focal consolidation . the cardiac silhouette is within normal limits . [ MASK ] opacity in the left midlung overlying the posterior left 5th rib may represent focal airspace disease . increased opacity in the right upper lobe with [ MASK ] associated atelectasis may represent focal consolidation or mass lesion with atelectasis . recommend chest ct for further evaluation . [ MASK ] opacity overlying the left 5th rib may represent focal airspace disease .',
  'entities': {'1': {'tokens': 'increased',
    'label': 'Observation::definitely present',
    'start_ix': 5,
    'end_ix': 5,
    'relations': [['modify', '2']]},
   '2': {'tokens': 'opacity',
    'label': 'Observation::definitely present',
    'start_ix': 6,
    'end_ix': 6,
    'relations': [['located_at', '5'],
     ['suggestive_of', '6'],
     ['suggestive_of', '7'],
     ['suggestive_of', '8']

In [59]:
attributes = []

for i in range(len(most_common_attributes)):
    attributes.append(most_common_attributes[i][0])

In [60]:
attributes

['cardiomegaly',
 'atelectasis',
 'opacities',
 'disease',
 'opacity',
 'limits',
 'normal limits',
 'granuloma',
 'degenerative changes',
 'effusions',
 'effusion',
 'emphysema',
 'changes',
 'markings',
 'granulomas',
 'calcifications',
 'midline',
 'density',
 'tortuosity',
 'sternotomy',
 'hiatal hernia',
 'lymph',
 'prominence',
 'deformity',
 'calcification',
 'osteophytes',
 'elevation',
 'edema',
 'surgical clips',
 'congestion',
 'nodules',
 'pneumonia',
 'enlargement',
 'clips',
 'hyperexpansion',
 'scoliosis',
 'copd',
 'dextroscoliosis',
 'calcified granuloma',
 'eventration',
 'pneumothorax',
 'tip',
 'consolidation',
 'aeration',
 'lucency',
 'ectasia',
 'deformities',
 'mild degenerative changes',
 'loss',
 'emphysematous changes']

In [61]:
def split_into_sentences(text):
    # Tokenize the text into sentences
    doc = nlp(text)

    # Extract sentences
    sentences = [sent.text for sent in doc.sents]

    return sentences

In [62]:
train_df["sentences"] = train_df["text"].apply(split_into_sentences)

In [63]:
val_df["sentences"] = val_df["text"].apply(split_into_sentences)

In [64]:
train_df.iloc[203]['sentences']

['hyperinflated lungs with flattened diaphragm and increased retrosternal airspace .',
 'heart size within normal limits .',
 'right hilar calcification suggests a previous granulomatous process .',
 'hyperinflated lungs , air trapping versus inspiratory [MASK] .']

In [65]:
for attribute in attributes:
    # Split the attribute into individual words
    attribute_words = attribute.split()
    
    # Identify sentences that contain any word from the attribute
    sentences_with_attributes = []

    for sentence_list in val_df["sentences"]:
        if sentence_list:
            for sentence in sentence_list:
                # Check if any word from attribute_words is in the sentence
                if any(word in sentence for word in attribute_words):
                    sentences_with_attributes.append(sentence)

    # Count the frequency of each sentence containing the attribute
    sentence_counter = Counter(sentences_with_attributes)

    # Get the 50 most frequent sentences
    most_common_sentences = [
        sentence for sentence, _ in sentence_counter.most_common(50)
    ]

    # Print the attribute and the number of matched sentences for debugging
    print(f"Attribute: {attribute} with {len(most_common_sentences)} sentences")
    print(len(most_common_sentences))


Attribute: cardiomegaly with 49 sentences

49

Attribute: atelectasis with 50 sentences

50

Attribute: opacities with 50 sentences

...

50

Attribute: loss with 6 sentences

6

Attribute: emphysematous changes with 50 sentences

50

In [ ]:
# from medclip import MedCLIPProcessor, MedCLIPTextModel

# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Using device: {device}")

# attribute_embeddings = []

# processor = MedCLIPProcessor()
# model = MedCLIPTextModel()
# model.cuda()

# for attribute in attributes:
#     # Identify sentences that contain the attribute from the "sentences" column
#     sentences_with_attributes = []

#     for sentence_list in train_df["sentences"]:
#         if sentence_list:
#             for sentence in sentence_list:
#                 if attribute in sentence:
#                     sentences_with_attributes.append(sentence)

#     # Count the frequency of each sentence containing the attribute
#     sentence_counter = Counter(sentences_with_attributes)

#     # Get the 200 most frequent sentences
#     most_common_sentences = [
#         sentence for sentence, _ in sentence_counter.most_common(200)
#     ]

#     with torch.inference_mode():
#         if most_common_sentences:
#             # Compute sentence embeddings
#             most_common_sentences = processor(
#                 text=most_common_sentences, return_tensors="pt", padding=True
#             ).to("cuda")
#             sentence_embeddings = model.forward(
#                 most_common_sentences["input_ids"],
#                 most_common_sentences["attention_mask"],
#             )

#             # Average the embeddings
#             average_embedding = sentence_embeddings.mean(axis=0)
#             average_embedding = average_embedding.cpu().detach().numpy()
#             attribute_embeddings.append(average_embedding)
#         else:
#             print(f"No sentences found for attribute: {attribute}")

# attribute_embeddings = np.stack(attribute_embeddings)
# attribute_embeddings = torch.tensor(attribute_embeddings).to(device)
# print(attribute_embeddings.shape)
# attribute_embeddings = dict(zip(attributes, attribute_embeddings))

# # Output the averaged embeddings
# for attribute, embedding in attribute_embeddings.items():
#     print(f"Shape: {embedding.shape}")
#     print(f"Attribute: {attribute}")
#     print(f"Embedding: {embedding[:10]}, dtype: {embedding.dtype}")

In [ ]:
# # Save embeddings
# torch.save(attribute_embeddings, f"{DATA_DIR}/attr_embs.pth")

In [66]:
frames = [train_df, val_df]

In [67]:
df = pd.concat(frames)

In [68]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes,split,sentences
2531,2531,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27.0, 14.0, 102.0, 160.0], [111.0, 14.0, 193...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the heart size is normal . the mediastinal con...,[multiple calcified granulomas],train,"[the heart size is normal ., the mediastinal c..."
1750,1750,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 12.0, 107.0, 205.0], [115.0, 14.0, 183...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size within normal limits .,[limits],train,[heart size within normal limits .]
3386,3386,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[18.0, 27.0, 104.0, 166.0], [115.0, 24.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"the heart , pulmonary [MASK] and mediastinum a...",[mild degenerative changes],train,"[the heart , pulmonary [MASK] and mediastinum ..."
187,187,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[14.0, 21.0, 103.0, 171.0], [115.0, 22.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,pa and moderate loss of the chest demonstrate ...,"[loss, ectasia, emphysematous changes, emphyse...",train,[pa and moderate loss of the chest demonstrate...
1231,1231,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27.0, 32.0, 121.0, 187.0], [130.0, 30.0, 215...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,moderate cardiomegaly . bibasilar and perihila...,"[cardiomegaly, opacities, cardiomegaly, edema]",train,"[moderate cardiomegaly ., bibasilar and perihi..."
...,...,...,...,...,...,...,...,...,...,...,...,...
541,541,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[37.0, 24.0, 118.0, 148.0], [128.0, 24.0, 201...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the cardiomediastinal silhouette is within nor...,[limits],val,[the cardiomediastinal silhouette is within no...
2899,2899,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[21.0, 26.0, 111.0, 169.0], [122.0, 30.0, 200...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the cardiomediastinal silhouette is within nor...,[normal limits],val,[the cardiomediastinal silhouette is within no...
790,790,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 47.0, 115.0, 185.0], [128.0, 40.0, 213...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the heart is normal in size . the mediastinal ...,"[mild prominence, prominence, lucent, prominence]",val,"[the heart is normal in size ., the mediastina..."
1021,1021,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 28.0, 114.0, 198.0], [121.0, 23.0, 206...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size is normal . lungs are clear . minim...,[atelectasis],val,"[heart size is normal ., lungs are clear ., mi..."


### Filter out attibutes that are not in the list of attributes


In [69]:
attributes

['cardiomegaly',
 'atelectasis',
 'opacities',
 'disease',
 'opacity',
 'limits',
 'normal limits',
 'granuloma',
 'degenerative changes',
 'effusions',
 'effusion',
 'emphysema',
 'changes',
 'markings',
 'granulomas',
 'calcifications',
 'midline',
 'density',
 'tortuosity',
 'sternotomy',
 'hiatal hernia',
 'lymph',
 'prominence',
 'deformity',
 'calcification',
 'osteophytes',
 'elevation',
 'edema',
 'surgical clips',
 'congestion',
 'nodules',
 'pneumonia',
 'enlargement',
 'clips',
 'hyperexpansion',
 'scoliosis',
 'copd',
 'dextroscoliosis',
 'calcified granuloma',
 'eventration',
 'pneumothorax',
 'tip',
 'consolidation',
 'aeration',
 'lucency',
 'ectasia',
 'deformities',
 'mild degenerative changes',
 'loss',
 'emphysematous changes']

In [70]:
df["attributes"] = df["attributes"].apply(
    lambda x: [attr for attr in x if attr in attributes]
)

In [71]:
# # Remove row if no attributes are less than 3
mask = df["attributes"].apply(lambda x: len(x) > 0)

df["attributes"] = df["attributes"].where(mask)

df = df.dropna(subset=["attributes"])

In [72]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,attributes,split,sentences
1750,1750,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 12.0, 107.0, 205.0], [115.0, 14.0, 183...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size within normal limits .,[limits],train,[heart size within normal limits .]
3386,3386,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[18.0, 27.0, 104.0, 166.0], [115.0, 24.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"the heart , pulmonary [MASK] and mediastinum a...",[mild degenerative changes],train,"[the heart , pulmonary [MASK] and mediastinum ..."
187,187,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[14.0, 21.0, 103.0, 171.0], [115.0, 22.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,pa and moderate loss of the chest demonstrate ...,"[loss, ectasia, emphysematous changes, emphyse...",train,[pa and moderate loss of the chest demonstrate...
1231,1231,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27.0, 32.0, 121.0, 187.0], [130.0, 30.0, 215...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,moderate cardiomegaly . bibasilar and perihila...,"[cardiomegaly, opacities, cardiomegaly, edema]",train,"[moderate cardiomegaly ., bibasilar and perihi..."
683,683,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[10.0, 25.0, 94.0, 164.0], [103.0, 20.0, 197....","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size or great lungs are clear . calcifie...,[granuloma],train,"[heart size or great lungs are clear ., calcif..."
...,...,...,...,...,...,...,...,...,...,...,...,...
541,541,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[37.0, 24.0, 118.0, 148.0], [128.0, 24.0, 201...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the cardiomediastinal silhouette is within nor...,[limits],val,[the cardiomediastinal silhouette is within no...
2899,2899,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[21.0, 26.0, 111.0, 169.0], [122.0, 30.0, 200...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the cardiomediastinal silhouette is within nor...,[normal limits],val,[the cardiomediastinal silhouette is within no...
790,790,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 47.0, 115.0, 185.0], [128.0, 40.0, 213...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the heart is normal in size . the mediastinal ...,"[prominence, prominence]",val,"[the heart is normal in size ., the mediastina..."
1021,1021,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 28.0, 114.0, 198.0], [121.0, 23.0, 206...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size is normal . lungs are clear . minim...,[atelectasis],val,"[heart size is normal ., lungs are clear ., mi..."


In [73]:
df = df[
    [
        "image_id",
        "image_size",
        "image_filepath",
        "region_coord",
        "region_bbox",
        "region_labels",
        "num_regions",
        "text_filepath",
        "text",
        "sentences",
        "attributes",
        "split",
    ]
]

In [74]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,sentences,attributes,split
1750,1750,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 12.0, 107.0, 205.0], [115.0, 14.0, 183...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size within normal limits .,[heart size within normal limits .],[limits],train
3386,3386,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[18.0, 27.0, 104.0, 166.0], [115.0, 24.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"the heart , pulmonary [MASK] and mediastinum a...","[the heart , pulmonary [MASK] and mediastinum ...",[mild degenerative changes],train
187,187,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[14.0, 21.0, 103.0, 171.0], [115.0, 22.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,pa and moderate loss of the chest demonstrate ...,[pa and moderate loss of the chest demonstrate...,"[loss, ectasia, emphysematous changes, emphyse...",train
1231,1231,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27.0, 32.0, 121.0, 187.0], [130.0, 30.0, 215...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,moderate cardiomegaly . bibasilar and perihila...,"[moderate cardiomegaly ., bibasilar and perihi...","[cardiomegaly, opacities, cardiomegaly, edema]",train
683,683,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[10.0, 25.0, 94.0, 164.0], [103.0, 20.0, 197....","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size or great lungs are clear . calcifie...,"[heart size or great lungs are clear ., calcif...",[granuloma],train
...,...,...,...,...,...,...,...,...,...,...,...,...
541,541,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[37.0, 24.0, 118.0, 148.0], [128.0, 24.0, 201...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the cardiomediastinal silhouette is within nor...,[the cardiomediastinal silhouette is within no...,[limits],val
2899,2899,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[21.0, 26.0, 111.0, 169.0], [122.0, 30.0, 200...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the cardiomediastinal silhouette is within nor...,[the cardiomediastinal silhouette is within no...,[normal limits],val
790,790,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 47.0, 115.0, 185.0], [128.0, 40.0, 213...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the heart is normal in size . the mediastinal ...,"[the heart is normal in size ., the mediastina...","[prominence, prominence]",val
1021,1021,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 28.0, 114.0, 198.0], [121.0, 23.0, 206...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size is normal . lungs are clear . minim...,"[heart size is normal ., lungs are clear ., mi...",[atelectasis],val


In [75]:
id_mapping = {
    id_value: idx for idx, id_value in enumerate(sorted(df["image_id"]), start=0)
}
# Map the original image_id to the new ordered index
df["image_id"] = df["image_id"].map(id_mapping)
df["image_id"] = range(len(df))
df.reset_index(inplace=True)
df.drop(columns=["index"], inplace=True)

In [76]:
df

,image_id,image_size,image_filepath,region_coord,region_bbox,region_labels,num_regions,text_filepath,text,sentences,attributes,split
0,0,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 12.0, 107.0, 205.0], [115.0, 14.0, 183...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size within normal limits .,[heart size within normal limits .],[limits],train
1,1,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[18.0, 27.0, 104.0, 166.0], [115.0, 24.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"the heart , pulmonary [MASK] and mediastinum a...","[the heart , pulmonary [MASK] and mediastinum ...",[mild degenerative changes],train
2,2,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[14.0, 21.0, 103.0, 171.0], [115.0, 22.0, 197...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,pa and moderate loss of the chest demonstrate ...,[pa and moderate loss of the chest demonstrate...,"[loss, ectasia, emphysematous changes, emphyse...",train
3,3,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[27.0, 32.0, 121.0, 187.0], [130.0, 30.0, 215...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,moderate cardiomegaly . bibasilar and perihila...,"[moderate cardiomegaly ., bibasilar and perihi...","[cardiomegaly, opacities, cardiomegaly, edema]",train
4,4,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[10.0, 25.0, 94.0, 164.0], [103.0, 20.0, 197....","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size or great lungs are clear . calcifie...,"[heart size or great lungs are clear ., calcif...",[granuloma],train
...,...,...,...,...,...,...,...,...,...,...,...,...
1961,1961,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[37.0, 24.0, 118.0, 148.0], [128.0, 24.0, 201...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the cardiomediastinal silhouette is within nor...,[the cardiomediastinal silhouette is within no...,[limits],val
1962,1962,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[21.0, 26.0, 111.0, 169.0], [122.0, 30.0, 200...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the cardiomediastinal silhouette is within nor...,[the cardiomediastinal silhouette is within no...,[normal limits],val
1963,1963,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[35.0, 47.0, 115.0, 185.0], [128.0, 40.0, 213...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,the heart is normal in size . the mediastinal ...,"[the heart is normal in size ., the mediastina...","[prominence, prominence]",val
1964,1964,"[224, 224]",C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[33.0, 28.0, 114.0, 198.0], [121.0, 23.0, 206...","[[1], [2], [3]]",3,C:/Users/DryLab/Desktop/ConVIRT/Dataset/open-i...,heart size is normal . lungs are clear . minim...,"[heart size is normal ., lungs are clear ., mi...",[atelectasis],val


In [77]:
df.to_feather(f"{DATA_DIR}/annotations.feather")